In [196]:
# Import required libraries
import re
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [198]:
# Import Dataset
df = pd.read_csv("E:\MJ-DEV\Github\Course-Materials\\2) Data PreProcessing\dataset\Employee-DataBase.csv")
df.head()

,EmpID,EmpName,Designation,Business Unit,Gender,Nationality,Age,Mail ID,Mobile Number,On-Boarded,Billable,Salary,Hike,Hike %,Resignation
0,1004265,David,Developer,Dev Team,M,US,26,David265@gmail.com,(+91)-96xxxxxx10,02-04-2018,Y,450000,N,0,YES
1,1004266,John,Lead,Dev Team,M,US,30,John266@gmail.com,(+91)-69xxxxxx11,09-05-2018,N,900000,Y,10,YES
2,1004267,Abraham,Developer,Dev Team,M,US,27,NaN,(+91)-69xxxxxx12,15-06-2018,Y,600000,Y,13,NO
3,1004268,Lena,Manager,Pre Sales,F,NaN,55,Lena268@gmail.com,(+91)-69xxxxxx13,11-07-2018,Y,1200000,Y,6,NO
4,1004269,Nathan,Software Engineer,Dev Team,M,US,30,Nathan269@gmail.com,(+91)-69xxxxxx14,03-08-2018,Y,490000,Y,8,YES


# Prep Work

In [185]:
# Rename columnnames to small case
def column_cleansing(col):
    col = col.lower()
    col = re.sub("[^a-zA-Z0-9]+", "_", col)
    return col

print("Columns before transforming: ", df.columns)

df.columns = [ column_cleansing(col) for col in df.columns]
print("Columns after transforming: ", df.columns)

Columns before transforming:  Index(['EmpID', 'EmpName', 'Designation', 'Business Unit', 'Gender',
       'Nationality', 'Age', 'Mail ID', 'Mobile Number', 'On-Boarded',
       'Billable', 'Salary', 'Hike', 'Hike %', 'Resignation'],
      dtype='object')
Columns after transforming:  Index(['empid', 'empname', 'designation', 'business_unit', 'gender',
       'nationality', 'age', 'mail_id', 'mobile_number', 'on_boarded',
       'billable', 'salary', 'hike', 'hike_', 'resignation'],
      dtype='object')


# Check Redundent Data

In [186]:
# Print Number of duplicate records in dataframe
"""
At this Stage, Pandas not considering rows with null values, due to this you might not able to see full records. 
So this is not the place to perform this task."""
df.groupby(df.columns.tolist(), as_index=False).size()

,empid,empname,designation,business_unit,gender,nationality,age,mail_id,mobile_number,on_boarded,billable,salary,hike,hike_,resignation,size
0,1004265,David,Developer,Dev Team,M,US,26,David265@gmail.com,(+91)-96xxxxxx10,02-04-2018,Y,450000,N,0,YES,1
1,1004266,John,Lead,Dev Team,M,US,30,John266@gmail.com,(+91)-69xxxxxx11,09-05-2018,N,900000,Y,10,YES,1
2,1004269,Nathan,Software Engineer,Dev Team,M,US,30,Nathan269@gmail.com,(+91)-69xxxxxx14,03-08-2018,Y,490000,Y,8,YES,2
3,1004270,Thompson,Developer,Dev Team,M,US,24,Thompson270@gmail.com,(+91)-69xxxxxx15,30-10-2018,Y,500000,Y,10,NO,1
4,1004271,Richard,Developer,Dev Team,M,US,25,Richard271@gmail.com,(+91)-69xxxxxx16,16-11-2018,N,520000,Y,12,NO,1
5,1004273,Aby,UI/UX,Pre Sales,F,US,30,Aby273@gmail.com,(+91)-69xxxxxx18,26-11-2018,Y,500000,Y,20,NO,1
6,1004274,Mike,Manager,Pre Sales,M,US,40,Mike274@gmail.com,(+91)-69xxxxxx19,07-03-2018,Y,1100000,N,0,NO,1
7,1004277,Robert,BA,Pre Sales,M,US,38,Robert277@gmail.com,(+91)-69xxxxxx22,28-06-2018,N,450000,Y,23,YES,1
8,1004278,Lucy,Manager,Dev Team,F,US,41,Lucy278@gmail.com,(+91)-69xxxxxx23,28-08-2018,Y,1500000,Y,4,NO,1
9,1004279,Shawn,BA,Pre Sales,M,US,39,Shawn279@gmail.com,(+91)-69xxxxxx24,02-09-2018,Y,700000,N,0,NO,1


# Handle Missing Data

**Step:1** *Identify Null (or) NaN values row wise*

In [187]:
# df.shape[1] - df.count(axis=1)
df.isnull().sum(axis=1)

0     0
1     0
2     1
3     1
4     0
5     0
6     0
7     1
8     0
9     0
10    1
11    1
12    0
13    1
14    0
15    0
16    0
17    1
18    0
19    1
20    0
21    0
dtype: int64

**Step:2** *Identify Null (or) NaN values column wise*

In [188]:
# df.isnull().sum()
# df.count()
# for col in df.columns:
#     cnt = df[col].isnull().sum()
#     if cnt >= 1:
#         print(col, ': ', cnt)

**Step:3** *Identify Null (or) NaN values on whole dataframe with more info*

In [189]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   empid          22 non-null     int64 
 1   empname        22 non-null     object
 2   designation    22 non-null     object
 3   business_unit  22 non-null     object
 4   gender         22 non-null     object
 5   nationality    17 non-null     object
 6   age            22 non-null     int64 
 7   mail_id        19 non-null     object
 8   mobile_number  22 non-null     object
 9   on_boarded     22 non-null     object
 10  billable       22 non-null     object
 11  salary         22 non-null     int64 
 12  hike           22 non-null     object
 13  hike_          22 non-null     int64 
 14  resignation    22 non-null     object
dtypes: int64(4), object(11)
memory usage: 2.7+ KB


In [190]:
# Replace Missing Values for integer object

df_target = df[["resignation"]]
df_obj = df[[col for col in df.columns if df[col].dtype == "object" and col not in df_target.columns]]
imp_obj = SimpleImputer(missing_values=np.NaN, strategy="most_frequent")
df_obj = pd.DataFrame(imp_obj.fit_transform(df_obj), columns=df_obj.columns)

# Check Empty data
# print(df_obj.isnull().sum())

df_num = df[[col for col in df.columns if df[col].dtype != "object" and col not in df_target.columns]]
imp_num = SimpleImputer(missing_values=np.NaN, strategy="mean")
df_num = pd.DataFrame(imp_num.fit_transform(df_num), columns=df_num.columns)

# Check Empty data
# print(df_num.isnull().sum())

df = pd.concat([df_obj, df_num, df_target], axis=1)

# Handle Categorical Data

In [191]:
# Replace Categorical value with actual value: INDEPENDENT VARIABLE
cat_columns = ['designation', 'business_unit', 'gender', 'billable', 'hike']

df = pd.get_dummies(data=df, columns=cat_columns, drop_first=True)
df['resignation'] = df.pop('resignation')

# Replace catagorical value with actual value: DEPENDENT VARIABLE
df["resignation"] = lbl.fit_transform(df["resignation"])

df.head()

,empname,nationality,mail_id,mobile_number,on_boarded,empid,age,salary,hike_,designation_BackEnd Developer,designation_Developer,designation_Lead,designation_Manager,designation_Software Engineer,designation_UI/UX,business_unit_Pre Sales,gender_M,billable_Y,hike_Y,resignation
0,David,US,David265@gmail.com,(+91)-96xxxxxx10,02-04-2018,1004265.0,26.0,450000.0,0.0,0,1,0,0,0,0,0,1,1,0,1
1,John,US,John266@gmail.com,(+91)-69xxxxxx11,09-05-2018,1004266.0,30.0,900000.0,10.0,0,0,1,0,0,0,0,1,0,1,1
2,Abraham,US,David265@gmail.com,(+91)-69xxxxxx12,15-06-2018,1004267.0,27.0,600000.0,13.0,0,1,0,0,0,0,0,1,1,1,0
3,Lena,US,Lena268@gmail.com,(+91)-69xxxxxx13,11-07-2018,1004268.0,55.0,1200000.0,6.0,0,0,0,1,0,0,1,0,1,1,0
4,Nathan,US,Nathan269@gmail.com,(+91)-69xxxxxx14,03-08-2018,1004269.0,30.0,490000.0,8.0,0,0,0,0,1,0,0,1,1,1,1


# Feature Selection - Stage: 1

In [192]:
# Remove Redundent/Unwanted Information
df = df.drop(['empname', 'mail_id', 'mobile_number', 'on_boarded', 'empid', 'nationality'], axis=1)
df = df.drop_duplicates()
df.head()

,age,salary,hike_,designation_BackEnd Developer,designation_Developer,designation_Lead,designation_Manager,designation_Software Engineer,designation_UI/UX,business_unit_Pre Sales,gender_M,billable_Y,hike_Y,resignation
0,26.0,450000.0,0.0,0,1,0,0,0,0,0,1,1,0,1
1,30.0,900000.0,10.0,0,0,1,0,0,0,0,1,0,1,1
2,27.0,600000.0,13.0,0,1,0,0,0,0,0,1,1,1,0
3,55.0,1200000.0,6.0,0,0,0,1,0,0,1,0,1,1,0
4,30.0,490000.0,8.0,0,0,0,0,1,0,0,1,1,1,1


# Normalizing dominant data

In [193]:
sclr = StandardScaler()
df["salary"] = sclr.fit_transform(df["salary"].values.reshape(-1, 1))
df["age"] = sclr.fit_transform(df["age"].values.reshape(-1, 1))
df["hike_"] = sclr.fit_transform(df["hike_"].values.reshape(-1, 1))
df.head()

,age,salary,hike_,designation_BackEnd Developer,designation_Developer,designation_Lead,designation_Manager,designation_Software Engineer,designation_UI/UX,business_unit_Pre Sales,gender_M,billable_Y,hike_Y,resignation
0,-0.936259,-0.859805,-1.171333,0,1,0,0,0,0,0,1,1,0,1
1,-0.384664,0.684593,0.202453,0,0,1,0,0,0,0,1,0,1,1
2,-0.798360,-0.345006,0.614588,0,1,0,0,0,0,0,1,1,1,0
3,3.062800,1.714192,-0.347062,0,0,0,1,0,0,1,0,1,1,0
4,-0.384664,-0.722526,-0.072304,0,0,0,0,1,0,0,1,1,1,1


# Split dataframe to train and test sets

In [199]:
# Create X and y Variable
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Split dataset to train and test sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)